In [1]:
# import org.apache.spark.sql.cassandra._

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from Activity import Activity
from activityByProduct import activityByProduct
from pyspark.streaming.kafka import KafkaUtils
import os

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'

In [4]:
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 
sqlcontext=SQLContext(sc)

In [5]:
# conf.setAppName("Spark Cassandra")

In [6]:
# conf.set("spark.cassandra.connection.host","localhost")

In [7]:
# # sc = SparkContext(appName="PythonSparkStreamingKafka")
# sc = SparkContext(conf=conf)

# sc.setLogLevel("WARN")

In [8]:
# sqlcontext = SQLContext(sc)

In [9]:
# from cassandra.cluster import Cluster

In [10]:
# cluster = Cluster()

In [11]:
# session = cluster.connect('lambda')

In [12]:
# rows = session.execute("SELECT * FROM lambda.emp;")

In [13]:
# rows.column_types

In [14]:
ssc = StreamingContext(sc,2)

In [15]:
textDStream1 = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'data_pipeline':1})
textDStream = textDStream1.map(lambda x: x[1])

In [16]:
def subprocess(line):
    records = line.split("\t")
    if records[0] != "action" and len(records) == 7:
        return Activity(records[0],records[1],records[2],records[3],records[4],records[5],records[6])
    else:
        return None

In [17]:
def process(rdd):
    if not rdd.isEmpty():
        newRDD = rdd.map(subprocess)
    else:
        newRDD = None
    return newRDD

In [18]:
def process_query(rdd):
    if not rdd.isEmpty():
        newDF=rdd.map(lambda x: (x, )).toDF()
        newDF2 = newDF.select('_1.*')
        newDF2.createOrReplaceTempView("Activity")
#         toHDFS = sqlcontext.sql("select * from activity")
# #         toHDFS.write.mode("append").csv("hdfs://localhost:9000/kafka_spark",header=True)
#         toHDFS2 = toHDFS.select(['action', 'page', 'product', 'referrer', 'timestamp_hour', 'visitor'])
        visitorsByProduct = sqlcontext.sql("SELECT product, timestamp_hour, COUNT(DISTINCT visitor) as unique_visitors FROM Activity GROUP BY product, timestamp_hour")
        visitorsByProduct.show(10)
        visitorsByProduct.printSchema()
        visitorsByProduct.fillna()
        visitorsByProduct.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="stream_visitors_by_product_text", keyspace="lambda").save()

In [19]:
newDStream = textDStream.transform(process)

In [20]:
newDStream.foreachRDD(process_query)

In [21]:
# newDStream.pprint()

In [22]:
ssc.start()

In [23]:
# ssc.stop(stopSparkContext=False,stopGraceFully=True)

In [24]:
# df = sqlcontext.read.csv('hdfs://localhost:9000/kafka_spark/part-00000-00535f69-81ae-4191-bda1-fbcd66209aaf-c000.csv', header=True, inferSchema=True)

In [25]:
# df.printSchema()

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Banquet,Brown 'N ...| 1521807788003|              1|
|Ziploc,Double Zip...| 1521807815011|              1|
|Hot Shot,Flying I...| 1521807788963|              1|
|Purell,Advanced H...| 1521807808291|              1|
|Heinz,Distilled W...| 1521807799523|              1|
|Kroger,Dry Roaste...| 1521807822691|              1|
|Cloud 9,Cutlery C...| 1521807821251|              1|
|Farmland,Lower So...| 1521807805891|              1|
|Molton Brown,Re-C...| 1521807795811|              1|
|Aveeno,Men's Afte...| 1521807804323|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Rosewill,Hard Dri...| 1521807828820|              1|
|Gatorade,Fierce G...| 1521807830260|              1|
|De Cecco,Penne Ri...| 1521807828340|              1|
|Gunnar,Vayper Ony...| 1521807824020|              1|
|         OYO,Whiskey| 1521807811060|              1|
|Glass Plus,Glass ...| 1521807822100|              1|
|Comet,Comet With ...| 1521807825460|              1|
|La Tourangelle,Gr...| 1521807811540|              1|
|Andersen's,Creamy...| 1521807822100|              1|
|HDX,Table Top Air...| 1521807818260|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|     Kraft,Cool Whip| 1521807836634|              1|
|Clean & Clear,Ess...| 1521807830394|              1|
|Band-Aid,Flexible...| 1521807834714|              1|
|         OYO,Whiskey| 1521807834714|              1|
|Glad,Odor Shield ...| 1521807839514|              1|
|Sol Republic,JAX ...| 1521807838074|              1|
|Arm & Hammer,Ultr...| 1521807838554|              1|
|Glass Plus,Glass ...| 1521807832794|              1|
|e.l.f.,Smudge Pot...| 1521807830394|              1|
|CVS Total Home,"A...| 1521807839514|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Reynolds,Parchmen...| 1521807851118|              1|
|McCormick,Whole B...| 1521807850638|              1|
|Pond's,Cold Cream...| 1521807855918|              1|
|Trident,Spearmint...| 1521807854958|              1|
|Santa Sweets,Grap...| 1521807848718|              1|
|CVS Total Home,"A...| 1521807851598|              1|
|      Kroger,2% Milk| 1521807849198|              1|
|Kettle,Classic Ba...| 1521807852558|              1|
|Sol Republic,JAX ...| 1521807854958|              1|
|Crest,Pro-Health ...| 1521807865998|              1|
+--------------------+--------------+---------------+
only showing top 10 rows

root
 |-- product: string (nullable = true)
 |-- timestamp_hour: string (nullable = true)
 |-- unique_visitors: long (nullable = false)

+--------------------+--------------+---------------+
|             product|times